In [1]:
cd C:\Users\LIMDAESUN\Historical_Simulation

C:\Users\LIMDAESUN\Historical_Simulation


In [2]:
import numpy as np
import pandas as pd
from arch import arch_model
from scipy.optimize import minimize
import statsmodels.api as sm
import warnings
warnings.filterwarnings(action = 'ignore')

In [3]:
Price = pd.read_excel('코스피지수.xlsx', parse_dates= ['Date']).set_index('Date')
my_ret = Price.pct_change().rename( columns = {Price.columns[0] : '코스피수익률'}).iloc[1:]
ret = my_ret[my_ret.columns[0]].dropna()

In [4]:
q = 0.99

 ### ARCH11
 $$
\begin{array}{c}
\sigma_t^2 = \omega + \alpha_1 u_{t-1}^2
\\
\hat{z}_{t-\tau} = \frac{R_{t-\tau}}{\sigma_{t-\tau}} 
\end{array}
$$
 ### GARCH11
 $$
\begin{array}{c}
\sigma_t^2 = \omega + \alpha u_{t}^2 + \beta \sigma_{t-1}^2
\\
\hat{z}_{t-\tau} = \frac{R_{t-\tau}}{\sigma_{t-\tau}} 
\end{array}
$$


In [5]:
class VaR_FHS :
    def ARCH11(ret,q = 0.99, max_lag = 252, ES_result = False) :
        am1 = arch_model(ret[-max_lag:], p=1, q=0) 
        res1 = am1.fit()
        Z = ret/res1.conditional_volatility
        R = pd.Series(Z * res1.conditional_volatility[-1])
        VaR = R.quantile(1-q)
        if ES_result == False :
            return VaR
        else :
            ES = R[R<=VaR].mean()
            return VaR, ES
    def GARCH11(ret, q = 0.99, max_lag = 252, ES_result = False) :
        gam11 = arch_model(ret[-max_lag:], p=1, q=1)
        resg11 = gam11.fit()
        Z = ret/resg11.conditional_volatility
        R = pd.Series(Z * resg11.conditional_volatility[-1])
        VaR = R.quantile(1-q)
        if ES_result == False :
            return VaR
        else :
            ES = R[R<=VaR].mean()
            return VaR, ES

 ### Weighted Historical Simulation
 $$
\begin{array}{c}
\omega_{\tau} = \frac{1-\lambda}{1-\lambda^m} \lambda^{\tau-1} \\ 
(if \ \tau \ is \ 1, \ the \ most \ recent \ wegiht ) \\ 
( m \ is \ total \ observation )
\end{array}
$$


In [6]:
def VaR_WHS(ret, lamb = 0.97, ES_result = False) :
    m = len(ret)
    ret = pd.Series(ret)
    tau = np.arange(1,m+1)
    w = pd.Series((lamb**(tau-1) * (1-lamb) / (1-lamb**m))[::-1], index = pd.Series(ret).index)
    data = pd.concat( [ret, w], axis = 1)
    data.columns = ['ret','w']
    sorted_data = data.sort_values(by  = [data.columns[0]])
    sorted_data['cum_w'] = sorted_data.w.cumsum(0)
    n = np.abs(np.array(sorted_data.cum_w) - (1-q)).argmin()
    if sorted_data.cum_w.iloc[n] == np.round(1-q,4) :
        VaR = sorted_data.ret.iloc[n]
    else :
        my_data = sorted_data.iloc[n-1:n+2].set_index('cum_w')['ret']
        temp = pd.Series([np.nan], index = [1-q])
        VaR = pd.concat([temp,my_data], axis = 0).sort_index().interpolate(method = 'linear').loc[1-q]
    if ES_result == False :
        return VaR
    else :
        ES = ret[ret<=VaR].mean()
        return VaR, ES

In [11]:
VaR_FHS.ARCH11(ret, ES_result=True)

Iteration:      1,   Func. Count:      5,   Neg. LLF: -183.9798530338926
Iteration:      2,   Func. Count:     15,   Neg. LLF: -184.1134694657771
Iteration:      3,   Func. Count:     23,   Neg. LLF: -190.0486892618782
Iteration:      4,   Func. Count:     28,   Neg. LLF: -190.47651893925746
Iteration:      5,   Func. Count:     34,   Neg. LLF: -191.33429165226232
Iteration:      6,   Func. Count:     41,   Neg. LLF: -191.39047141691037
Iteration:      7,   Func. Count:     46,   Neg. LLF: -191.73041382207907
Iteration:      8,   Func. Count:     51,   Neg. LLF: -191.76207511177253
Iteration:      9,   Func. Count:     56,   Neg. LLF: -191.76798795167022
Iteration:     10,   Func. Count:     61,   Neg. LLF: -191.76808017629475
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -191.76808105560258
            Iterations: 10
            Function evaluations: 62
            Gradient evaluations: 10


(-0.12201322713431711, -0.17704387963866114)

In [12]:
VaR_FHS.GARCH11(ret, ES_result= True)

Iteration:      1,   Func. Count:      6,   Neg. LLF: -213.37767852193053
Iteration:      2,   Func. Count:     17,   Neg. LLF: -213.39491054294922
Iteration:      3,   Func. Count:     27,   Neg. LLF: -213.4054600995695
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -213.40546184007113
            Iterations: 7
            Function evaluations: 27
            Gradient evaluations: 3


C:\Users\LIMDAESUN\anaconda3\lib\site-packages\arch\univariate\base.py:709: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning,


(-0.07660381504649233, -0.09285053303165956)

In [13]:
VaR_WHS(ret, lamb = 0.98, ES_result=True)

(-0.10713640076444142, -0.16453438514442212)